Project made possible by following the guides:
-https://developer.twitter.com/en/docs/twitter-api/v1
-https://www.tweepy.org/
-https://datascienceparichay.com/article/python-get-data-from-twitter-api-v2/
-https://towardsdatascience.com/step-by-step-twitter-sentiment-analysis-in-python-d6f650ade58d
-https://towardsdatascience.com/twitter-sentiment-analysis-in-python-1bafebe0b566
-https://github.com/yogeshnile/Twitter-Sentiment-Analysis-on-Python-GUI

In [2]:
# Import Libraries

#need to install  python -m ipykernel install --user --name=twitter_venv  
#steps https://paulnelson.ca/posts/install-virtual-env-ipykernel-jupyter
# Tweepy - Python library for accessing the Twitter API.
import tweepy
# TextBlob - Python library for processing textual data
from textblob import TextBlob
# WordCloud - Python linrary for creating image wordclouds
from wordcloud import WordCloud, STOPWORDS 
# Pandas - Data manipulation and analysis library
import pandas as pd
#add table to GUI
from pandastable import Table, TableModel, config
# NumPy - mathematical functions on multi-dimensional arrays and matrices
import numpy as np
# Regular Expression Python module
import re
#GUI
from tkinter import *

# Matplotlib - plotting library to create graphs and charts
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
# Settings for Matplotlib graphs and charts
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8


from tqdm import tqdm


In [8]:
#Start of GUI
root = Tk()

#Title of Gui
root.title("Twitter Sentiment Analysis")

#GUI window size and banner
root.geometry("600x650")
banner = Frame(root, padx=1 ,pady=1, bg="red")
banner.pack()
heding = Label(banner, text="Twitter Sentiment Analysis", font="Arial 20 bold")
heding.pack()

#scroll bar
sb_ver = Scrollbar(root,orient=VERTICAL)
sb_ver.pack(side=RIGHT, fill=Y)
sb_ver.config(command=sb_ver.set)


#================User Input Part Begin ===================#
input_frame = Frame(root,padx=0,pady=30)
input_frame.pack(anchor="w")

input_frame1 = Frame(root,padx=0,pady=0, bg="yellow")
input_frame1.pack()

hash_value = StringVar()
hashtag = Label(input_frame, text="Enter phrase to search with # :- ",font="comicsansms 10 bold", padx=30).grid(row=4, column=1)
hashinput = Entry(input_frame, textvariable=hash_value).grid(row=4, column=2)

#================User Input Part End ===================#


#================Sentiment Part Begin===================#.
f1 = Frame(root,padx=15,pady=14)
f1.pack()

f2 = Frame(root,padx=15,pady=14)
f2.pack(anchor="w")

error = Label(f1, text="Please enter any one", fg="red")
error2 = Label(f1, text="Both entry not valid", fg="red")

#================Sentiment Part End===================#

po = Label(f2, text="Positive:-",padx=15)
na = Label(f2, text="Negative:-",pady=5,padx=15)
nt = Label(f2, text="Neutral:-",padx=15)
df_L = Label(f2, text=f"Tweets: ")

#for pie chart
frameChartsLT = Frame(root,)
frameChartsLT.pack()

    #============================Insert here twitter API keys===========================
    
def r_twitterApiKey():
    config = pd.read_csv("config.csv")
    twitterApiKey = config['consumer_key'][0]
    return twitterApiKey
def r_twitterApiSecret():
    config = pd.read_csv("config.csv")
    twitterApiSecret = config['consumer_secret'][0]
    return twitterApiSecret
def r_twitterApiAccessToken():
    config = pd.read_csv("config.csv")
    twitterApiAccessToken = config['access_token'][0]
    return twitterApiAccessToken
def r_twitterApiAccessTokenSecret():
    config = pd.read_csv("config.csv")
    twitterApiAccessTokenSecret = config['access_secret'][0]
    return twitterApiAccessTokenSecret
    #============================End twitter API keys section===========================

def click():  
    #user_name = user_value.get()
    hash_name = hash_value.get()
    
    # Authenticate
    auth = tweepy.OAuthHandler(r_twitterApiKey(), r_twitterApiSecret())
    auth.set_access_token(r_twitterApiAccessToken(), r_twitterApiAccessTokenSecret())
    api = tweepy.API(auth)
    
    if hash_name == "":
        error.grid_remove()
        global number
        if number > 1:
            po.grid_remove()
            na.grid_remove()
            nt.grid_remove()
            #tweet_label.grid_remove()
  
    msgs = []
    msg =[]
    #set number of tweets to return
    for tweet in tweepy.Cursor(api.search_tweets, q=hash_name,lang='en').items(500):
        msg = [tweet.text] 
        msg = tuple(msg)                    
        msgs.append(msg)
     

    def cleanTxt(text):
        text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
        text = re.sub('#', '', text) # Removing '#' hash tag
        text = re.sub('RT[\s]+', '', text) # Removing RT
        text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
        return text
    df = pd.DataFrame(msgs)
    df['Tweets'] = df[0].apply(cleanTxt)
    df.drop(0, axis=1, inplace=True)
    
    #calculate the subjectivity and polarity of tweets.
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity
    df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
    df['Polarity'] = df['Tweets'].apply(getPolarity)
    

    # negative, nautral, positive analysis
    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'
 
    #getting percentages from results
    df['Analysis'] = df['Polarity'].apply(getAnalysis)
    positive = df.loc[df['Analysis'].str.contains('Positive')]
    negative = df.loc[df['Analysis'].str.contains('Negative')]
    neutral = df.loc[df['Analysis'].str.contains('Neutral')]

    positive_per = round((positive.shape[0]/df.shape[0])*100, 1)
    negative_per = round((negative.shape[0]/df.shape[0])*100, 1)
    neutral_per = round((neutral.shape[0]/df.shape[0])*100, 1)

    #show sentiment percentage
    po = Label(f2, text=f"Positive:- {positive_per}%",padx=15).grid(row=1, column=2)
    na = Label(f2, text=f"Negative:- {negative_per}%",pady=5,padx=15).grid(row=2, column=2)
    nt = Label(f2, text=f"Neutral:- {neutral_per}%",padx=15).grid(row=3, column=2)
    number +=1

    #show tweets
    df_L = Label(f2, text=f"{df}%",padx=15).grid(row=5, column=1)
    #print(str(pos_p) + " % of positive tweets")
    #print(str(neg_p) + " % of negative tweets")
    #print(str(neu_p) + " % of neutral tweets")

    #assign to pie chart. instructions followed from: https://stackoverflow.com/questions/41088236/how-to-have-actual-values-in-matplotlib-pie-chart-displayed#:~:text=If%20you%27re%20looking%20to%20plot%20a%20piechart%20from,autopct%3D%20lambda%20x%3A%20%27%20%7B%3A.0f%7D%27.format%20%28x%2Avalues.sum%20%28%29%2F100%29%2C%20startangle%3D90%29
    
    fig = Figure(figsize = (3, 3),
                 dpi = 100) # create a figure object
    ax = fig.add_subplot(111) # add an Axes to the figure
    sizes = [positive_per, neutral_per, negative_per]
    colors = ['green','blue','red']

    legend_text  = ['Positive ['+str(positive_per)+'%]' , 'Neutral ['+str(neutral_per)+'%]','Negative ['+str(negative_per)+'%]']
    legend_frame = LabelFrame(text='Legend',padx=5, pady=5)
    legend_label = Label(legend_frame,text=legend_text)
    legend_label.pack()

    ax.pie(sizes,colors=colors, startangle=90)
    chart1 = FigureCanvasTkAgg(fig,frameChartsLT)
    chart1.get_tk_widget().pack()
    


#Begin
number=0
button = Button(input_frame1,text="Get Analysis", command=click, fg="blue",height = 1, width = 15)
button.grid(row=1, column=1)


# Function for closing window
exit_button = Button(root, text="Quit", command=root.destroy).pack()



root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\almag\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\almag\AppData\Local\Temp\ipykernel_11280\4193612049.py", line 96, in click
    for tweet in tweepy.Cursor(api.search_tweets, q=hash_name,lang='en').items(500):
  File "c:\Users\almag\AppData\Local\Programs\Python\Python310\lib\site-packages\tweepy\cursor.py", line 86, in __next__
    return self.next()
  File "c:\Users\almag\AppData\Local\Programs\Python\Python310\lib\site-packages\tweepy\cursor.py", line 286, in next
    self.current_page = next(self.page_iterator)
  File "c:\Users\almag\AppData\Local\Programs\Python\Python310\lib\site-packages\tweepy\cursor.py", line 86, in __next__
    return self.next()
  File "c:\Users\almag\AppData\Local\Programs\Python\Python310\lib\site-packages\tweepy\cursor.py", line 167, in next
    data = self.method(max_id=self.max_id, pa

In [ ]:
#Creating new data frames for all sentiments (positive, negative and neutral)
tw_list_negative = df[df["Score"]=="negative"]
tw_list_positive = df[df["Score"]=="positive"]
tw_list_neutral = df[df["Score"]=="neutral"]

#Creating new dataframe and new features dor word cloud
tweet_list = []

for tweet in tweets:
    #print(tweet.text)
    tweet_list.append(tweet.Tweet)

tweet_list = pd.DataFrame(tweet_list)
tweet_list["text"] = df['Tweet']
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list.text.str.lower()

#Function to Create Wordcloud
def create_wordcloud(text): 
    stopwords = set(STOPWORDS)
    wc = WordCloud(background_color="white",
    max_words=3000,
    stopwords=stopwords,
    repeat=True)
    pa = wc.generate(str(text))
    print("Word Cloud Saved Successfully")
    plt.imshow(pa)
    plt.axis('off')
    plt.show()

#Creating wordcloud for all tweets
create_wordcloud(positive.values)

#Creating wordcloud for positive sentiment
create_wordcloud(negative.values)

#Creating wordcloud for positive sentiment
create_wordcloud(neutral.values)